<a href="https://colab.research.google.com/github/LanceMeister/SEO-Audit-Google-Search-Console-for-SEOs/blob/main/SEO_Audit_Google_Search_Console_Machine_Learning_for_SEOs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# A Hands-on Introduction to Machine Learning for SEOs

Here is technical plan to generate the training dataset:

1. Extract: we will walk through my code to connect to Google Search Console and pull CTR data
2. Transform: then, we will fetch the pages titles and meta descriptions and normalize/clean them
3. Load: finally, we will populate a Google Sheet with the pages, titles, meta descriptions and CTRs

In [ ]:
spreadsheetName="GSC Data" #@param {type:"string"}
sitename="sc-domain:meisteraffiliateshop.com" #@param {type:"string"}
client_id='google_secret.json' #@param {type:"string"}


## Extracting from Google Search Console

First, there is some setup to download a client_id.json file our Python code can use to connect securely to Google Search Console.

1. Activate Search Console API in Compute Engine  https://console.cloud.google.com/apis/api/webmasters.googleapis.com/overview?project=&folder=&organizationId=
2. Create New Credentials / Help me choose (Search Console API, Other UI, User data)  https://console.cloud.google.com/apis/credentials/wizard?api=iamcredentials.googleapis.com&project=
3. Download client_id.json

In [ ]:
#@title
# Go to Runtime > run after
!pip install git+https://github.com/joshcarty/google-searchconsole

In [ ]:
#@title
# run once
import searchconsole
#Create the account connection. 
try:
	account = searchconsole.authenticate(flow="console",client_config='/content/drive/MyDrive/config/client_secret_395713215546-kmpj3c3r57e5f08dmrmd36js8u2m5av6.apps.googleusercontent.com.json',credentials='/content/drive/MyDrive/config/buoyant-zodiac-328808-e9f3add2cb89.json')
except:
	account = searchconsole.authenticate(flow="console",client_config='/content/drive/MyDrive/config/client_secret_395713215546-kmpj3c3r57e5f08dmrmd36js8u2m5av6.apps.googleusercontent.com.json',serialize='/content/drive/MyDrive/config/buoyant-zodiac-328808-e9f3add2cb89.json')



Let's get our Search Console data

In [ ]:
#@title
webproperty = account[sitename]

In [ ]:
#@title
#let's build a pandas dataframe with the search console data
import pandas as pd
%load_ext google.colab.data_table

def get_search_console_data(webproperty, days=-7):
  if webproperty is not None:
    query = webproperty.query.range(start='today', days=days).dimension('date', 'page', 'query', 'country', 'device')

    r = query.get()
    df = pd.DataFrame(r.rows)
    return df

  print("Web property doesn't exist, please select a valid one from this list")
  print(account.webproperties)

  return None


In [ ]:
#@title
df = get_search_console_data(webproperty)

In [ ]:
#@title
df.info()

In [ ]:
#@title
df.head()

In [ ]:
#@title
pages = list(set(df.page))
len(pages)

## Extracting SEO Audit

In [ ]:
#@title
try:
  from seoanalyzer import analyze
except:
  !pip install git+https://github.com/LanceMeister/python-seo-analyzer
  from seoanalyzer import analyze

In [ ]:
#@title
output = analyze('https://www.meisteraffiliateshop.com', analyze_headings=True, analyze_extra_tags=True) 
seo_df= pd.DataFrame(output['pages'])
seo_df.head()

In [ ]:
#@title
seo_df.info()

In [ ]:
#@title
seo_df.dropna().head()

## Merge dataframes



In [ ]:
#@title
merged_df=pd.merge(df, seo_df, how="left", on="page")

In [ ]:
#@title
merged_df.head()

In [ ]:
#@title
merged_df.info()

In [ ]:
#@title
merged_df.dropna().head()

## Keywords in title

https://stackoverflow.com/questions/31806695/when-to-use-which-fuzz-function-to-compare-2-strings
https://chairnerd.seatgeek.com/fuzzywuzzy-fuzzy-string-matching-in-python/
https://github.com/seatgeek/fuzzywuzzy

In [ ]:
#@title
!pip install fuzzywuzzy[speedup]

In [ ]:
#@title
from fuzzywuzzy import fuzz

In [ ]:
#@title
#remove empty rows
df = merged_df.dropna()

In [ ]:
#@title
#df.apply(lambda row: fuzz.partial_ratio(row["query"], row["title"]), axis = 1)
df["query_in_title"] = df.apply(lambda row: fuzz.partial_ratio(row["query"], row["title"]), axis=1)
# shift column 'query_in_title' to tenth position
tenth_column = df.pop('query_in_title')
# insert column using insert(position,column_name,first_column) function
df.insert(10, 'query_in_title', tenth_column)
df

In [ ]:
#@title
de_df = df.query('country == "deu"')
de_df

## Populating our Google Sheet

https://pypi.org/project/gspread-pandas/

https://github.com/aiguofer/gspread-pandas/pull/47

https://gspread-pandas.readthedocs.io/en/latest/getting_started.html

In [ ]:
#@title
!pip install gspread-pandas
!pip install --upgrade google-auth[reauth]


In [ ]:
#@title
from google.colab import auth
auth.authenticate_user()
import google.auth
import gspread

In [ ]:
#@title
from gspread_pandas import Spread, Client

In [ ]:
#@title
import google.auth

creds, project = google.auth.default()


In [ ]:
#@title
spread = Spread(spreadsheetName, creds=creds)
spread


In [ ]:
#@title
# Display available worksheets
spread.sheets

[<Worksheet 'Sheet1' id:0>, <Worksheet 'Client' id:1120219873>]

In [ ]:
#@title
# Save DataFrame to worksheet 'Client', create it first if it doesn't exist
spread.df_to_sheet(df, index=False, sheet='Client', start='A1', replace=True)

In [ ]:
#@title
spread.sheets

[<Worksheet 'Sheet1' id:0>, <Worksheet 'Client' id:1120219873>]